# 2D Plane Strain FEA — CalculiX (Full Spring)

Industrial-grade FEA verification of the **complete doubly-clamped bistable spring**
using CalculiX (ccx) with `NLGEOM`.

**Geometry**: Two parallel CCS half-beam pairs + solid shuttle rectangle, merged
into a single polygon.

**Boundary conditions**:
- Left anchor (x≈0): clamped, u_x = u_y = 0
- Right anchor (x≈anchor_distance): clamped, u_x = u_y = 0
- Shuttle (center): prescribe u_y, u_x free

**Workflow**: Generate mesh with `triangle` → write `.inp` → run `ccx` →
parse reaction forces from `.dat` output.

Runs at 3 mesh densities (10%, 50%, 100%).

In [ ]:
import sys
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import triangle as tr
import shutil

sys.path.insert(0, os.path.dirname(os.path.abspath("__file__")))
from full_spring_utils import (
    POLY_SI, DEFAULT_FULL_SPRING_PARAMS,
    get_full_spring_polygon, identify_bc_nodes_2d,
)

P = DEFAULT_FULL_SPRING_PARAMS
anchor_dist = P['anchor_distance']
half_span = P['half_span']
shuttle_len = P['shuttle_length']
h = P['initial_offset']
t = POLY_SI['t']
E = POLY_SI['E']
nu = POLY_SI['nu']

CCX = shutil.which('ccx')
if CCX is None:
    CCX = os.path.abspath('../../switch-env/bin/ccx')
print(f"CalculiX: {CCX}")
result = subprocess.run([CCX, '-v'], capture_output=True, text=True)
print(result.stdout.strip())
print(f"Full spring: anchor_dist={anchor_dist} µm, half_span={half_span} µm")
print(f"  h={h} µm, t={t} µm")

os.makedirs('plots', exist_ok=True)
os.makedirs('results', exist_ok=True)

## 1. Geometry

In [ ]:
poly = get_full_spring_polygon(n_points=400)

# Prepare for triangle mesher
poly_open = poly.copy()
if np.allclose(poly_open[0], poly_open[-1]):
    poly_open = poly_open[:-1]

n_verts = len(poly_open)
segments = np.column_stack([np.arange(n_verts), (np.arange(n_verts) + 1) % n_verts])
tri_input = {'vertices': poly_open, 'segments': segments}

print(f"Polygon: {n_verts} vertices")

## 2. CalculiX Input Writer

In [ ]:
def write_nset(f, name, nids, per_line=16):
    """Write a node set to CalculiX .inp file."""
    f.write(f'*NSET, NSET={name}\n')
    for j, nid in enumerate(nids):
        f.write(f'{nid}')
        if (j + 1) % per_line == 0 or j == len(nids) - 1:
            f.write('\n')
        else:
            f.write(', ')


def write_ccx_2d_full(filepath, nodes, elems, left_nids, right_nids,
                       shuttle_nids, delta, E, nu, thickness):
    """Write CalculiX .inp for 2D plane strain full spring.
    
    BCs: left + right anchors clamped (u_x=u_y=0), shuttle y-prescribed.
    """
    with open(filepath, 'w') as f:
        f.write('*HEADING\n')
        f.write('CCS Full Bistable Spring - 2D Plane Strain\n')
        f.write('**\n')
        
        f.write('*NODE\n')
        for i, (x, y) in enumerate(nodes):
            f.write(f'{i+1}, {x:.8f}, {y:.8f}, 0.0\n')
        
        f.write('*ELEMENT, TYPE=CPE3, ELSET=SPRING\n')
        for i, (n1, n2, n3) in enumerate(elems):
            f.write(f'{i+1}, {n1+1}, {n2+1}, {n3+1}\n')
        
        write_nset(f, 'LEFT_ANCHOR', left_nids)
        write_nset(f, 'RIGHT_ANCHOR', right_nids)
        write_nset(f, 'SHUTTLE', shuttle_nids)
        
        f.write('*MATERIAL, NAME=POLYSI\n')
        f.write('*ELASTIC\n')
        f.write(f'{E:.1f}, {nu:.4f}\n')
        
        f.write('*SOLID SECTION, ELSET=SPRING, MATERIAL=POLYSI\n')
        f.write(f'{thickness:.4f}\n')
        
        f.write('*STEP, NLGEOM, INC=1000\n')
        f.write('*STATIC\n')
        f.write('0.05, 1.0, 1e-8, 0.2\n')
        
        # Left anchor: clamp
        f.write('*BOUNDARY\n')
        f.write('LEFT_ANCHOR, 1, 2, 0.0\n')
        
        # Right anchor: clamp
        f.write('*BOUNDARY\n')
        f.write('RIGHT_ANCHOR, 1, 2, 0.0\n')
        
        # Shuttle: prescribe y only (DOF 2)
        f.write('*BOUNDARY\n')
        f.write(f'SHUTTLE, 2, 2, {delta:.8f}\n')
        
        f.write('*NODE FILE\n')
        f.write('U, RF\n')
        
        f.write('*NODE PRINT, NSET=SHUTTLE, TOTALS=YES\n')
        f.write('RF\n')
        
        f.write('*END STEP\n')


def parse_ccx_reaction_force(dat_file):
    """Parse total reaction force (y-component) from .dat file."""
    with open(dat_file, 'r') as f:
        lines = f.readlines()
    rf_y = None
    for i, line in enumerate(lines):
        if 'total force' in line.lower():
            for j in range(i + 1, min(i + 5, len(lines))):
                parts = lines[j].split()
                if len(parts) >= 2:
                    try:
                        rf_y = float(parts[1])
                        break
                    except ValueError:
                        continue
    return rf_y

print("Input writer and parser ready.")

## 3. Displacement Sweep (3 densities)

In [ ]:
base_max_area = 0.008

densities = {
    '10pct': {'scale': 10.0, 'n_steps': 30, 'label': '10% density'},
    '50pct': {'scale': 2.0,  'n_steps': 40, 'label': '50% density'},
    '100pct': {'scale': 1.0, 'n_steps': 50, 'label': '100% density'},
}

all_results = {}

for density_name, cfg in densities.items():
    print(f"\n{'='*60}")
    print(f"Density: {cfg['label']} (scale={cfg['scale']})")
    print(f"{'='*60}")
    
    # Working directory
    ccx_dir = os.path.abspath(f'results/ccx_2d_full_{density_name}')
    os.makedirs(ccx_dir, exist_ok=True)
    
    # --- Mesh ---
    max_area = base_max_area * cfg['scale']
    tri_out = tr.triangulate(tri_input, f'pq30a{max_area}')
    nodes = tri_out['vertices']
    elems = tri_out['triangles']
    print(f"Mesh: {len(nodes)} nodes, {len(elems)} triangles")
    
    # --- Boundary nodes (1-indexed for CalculiX) ---
    bc = identify_bc_nodes_2d(nodes, anchor_dist, half_span, shuttle_len)
    left_nids = bc['left_anchor'] + 1
    right_nids = bc['right_anchor'] + 1
    shuttle_nids = bc['shuttle'] + 1
    print(f"Left anchor: {len(left_nids)}, Right anchor: {len(right_nids)}, Shuttle: {len(shuttle_nids)}")
    
    # --- Displacement sweep ---
    n_steps = cfg['n_steps']
    delta_vals = np.linspace(0, -2*h, n_steps)
    delta_vals[0] = -0.01  # small perturbation
    F_ccx = np.zeros(n_steps)
    
    for i, delta in enumerate(delta_vals):
        job = f'step_{i:03d}'
        inp_file = os.path.join(ccx_dir, f'{job}.inp')
        
        write_ccx_2d_full(inp_file, nodes, elems, left_nids, right_nids,
                          shuttle_nids, delta, E, nu, t)
        
        result = subprocess.run(
            [CCX, '-i', job],
            cwd=ccx_dir,
            capture_output=True, text=True, timeout=120
        )
        
        dat_file = os.path.join(ccx_dir, f'{job}.dat')
        if os.path.exists(dat_file):
            rf = parse_ccx_reaction_force(dat_file)
            if rf is not None:
                F_ccx[i] = rf
            else:
                print(f"  Step {i}: could not parse RF")
        else:
            print(f"  Step {i}: ccx failed (rc={result.returncode})")
        
        if i % 10 == 0:
            print(f"  Step {i}/{n_steps}: δ={delta:.3f} µm, F={F_ccx[i]:.4f} µN")
    
    # Fix first point
    delta_vals[0] = 0.0
    F_ccx[0] = 0.0
    
    print(f"Done. F range: [{F_ccx.min():.4f}, {F_ccx.max():.4f}] µN")
    
    all_results[density_name] = {
        'delta': -delta_vals,
        'F': F_ccx,
        'n_nodes': len(nodes),
        'n_elems': len(elems),
    }
    
    np.savetxt(f'results/calculix_2d_full_{density_name}.csv',
               np.column_stack([-delta_vals, F_ccx]),
               delimiter=',', header='delta_um,F_uN', comments='')
    print(f"Saved: results/calculix_2d_full_{density_name}.csv")

## 4. Mesh Convergence Plot

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

colors = {'10pct': 'green', '50pct': 'blue', '100pct': 'red'}
for name, res in all_results.items():
    label = f"{densities[name]['label']} ({res['n_nodes']} nodes, {res['n_elems']} elems)"
    ax.plot(res['delta'], res['F'], 'o-', color=colors[name], linewidth=1.5,
            ms=2, label=label)

try:
    ana = np.loadtxt('results/analytical_force_displacement.csv',
                     delimiter=',', skiprows=1)
    ax.plot(ana[:, 0], 4 * ana[:, 2], 'k:', linewidth=1.5,
            label='Analytical CCS (×4)')
except FileNotFoundError:
    pass

ax.axhline(0, color='gray', linewidth=0.5)
ax.set_xlabel('Shuttle displacement from initial [µm]')
ax.set_ylabel('Force F [µN]')
ax.set_title('CalculiX 2D Full Spring — Mesh Convergence')
ax.legend()
ax.grid(True, alpha=0.3)

fig.tight_layout()
fig.savefig('plots/calculix_2d_full_convergence.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Deformed Shapes (Fine Mesh)

In [ ]:
def parse_ccx_frd_displacements(frd_file, n_nodes):
    """Parse nodal displacements from CalculiX .frd output."""
    ux = np.zeros(n_nodes)
    uy = np.zeros(n_nodes)
    with open(frd_file, 'r') as f:
        lines = f.readlines()
    in_disp = False
    for line in lines:
        if 'DISP' in line or 'DISPR' in line:
            in_disp = True
            continue
        if in_disp:
            if line.startswith(' -3'):
                break
            if line.startswith(' -1'):
                parts = line.split()
                try:
                    nid = int(parts[1]) - 1
                    if 0 <= nid < n_nodes:
                        ux[nid] = float(parts[2])
                        uy[nid] = float(parts[3])
                except (IndexError, ValueError):
                    continue
    return ux, uy

# Use finest mesh
finest = '100pct' if '100pct' in all_results else list(all_results.keys())[-1]
res = all_results[finest]
n_ccx_steps = densities[finest]['n_steps']
ccx_dir = os.path.abspath(f'results/ccx_2d_full_{finest}')

# Re-mesh to get node coordinates
max_area = base_max_area * densities[finest]['scale']
tri_out = tr.triangulate(tri_input, f'pq30a{max_area}')
nodes_vis = tri_out['vertices']
elems_vis = tri_out['triangles']

fig, ax = plt.subplots(figsize=(16, 6))
ax.triplot(nodes_vis[:, 0], nodes_vis[:, 1], elems_vis, 'k-', alpha=0.1, linewidth=0.2)

key_steps = [0, n_ccx_steps//4, n_ccx_steps//2, 3*n_ccx_steps//4, n_ccx_steps-1]
cmap_colors = plt.cm.viridis(np.linspace(0, 1, len(key_steps)))

delta_sweep = np.linspace(0, -2*h, n_ccx_steps)
delta_sweep[0] = 0.0

for k, step_idx in enumerate(key_steps):
    frd_file = os.path.join(ccx_dir, f'step_{step_idx:03d}.frd')
    if os.path.exists(frd_file):
        ux, uy = parse_ccx_frd_displacements(frd_file, len(nodes_vis))
        ax.triplot(nodes_vis[:, 0] + ux, nodes_vis[:, 1] + uy, elems_vis,
                   '-', color=cmap_colors[k], linewidth=0.3,
                   label=f'δ={-delta_sweep[step_idx]:.2f} µm')

ax.set_aspect('equal')
ax.set_xlabel('x [µm]')
ax.set_ylabel('y [µm]')
ax.set_title(f'CalculiX 2D Full Spring — Deformed Shapes ({finest})')
ax.legend(fontsize=9)
fig.tight_layout()
fig.savefig('plots/calculix_2d_full_deformed.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Summary

In [ ]:
print(f"{'Density':15s} {'Nodes':>8s} {'Elems':>8s} {'F_max [µN]':>12s} {'F_min [µN]':>12s}")
print('-' * 60)
for name, res in all_results.items():
    print(f"{densities[name]['label']:15s} {res['n_nodes']:8d} {res['n_elems']:8d} "
          f"{res['F'].max():12.4f} {res['F'].min():12.4f}")

print(f"\nExpected: F_push ≈ 10 µN, snap-through at δ ≈ {h:.1f} µm")